# Wine Inventory

There are a collection of wines that have been cataloged by wines we have purchased and consumed over the past few years: "onShelf.csv"

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [132]:
# creating a single dataset of all the wines

headers = ['Location Code', 'Location', 'Vintage', 'Region', 'Wine Name', 'Grape/Blend', 'Quantity', 
           'Rating', 'Unit Purchase Price', 'Type', 'Purchase Date']

df = pd.read_csv('onShelf.csv', names=headers, header=0, index_col=False)

df = df.reset_index(drop=True)

In [133]:
df.head()

,Location Code,Location,Vintage,Region,Wine Name,Grape/Blend,Quantity,Rating,Unit Purchase Price,Type,Purchase Date
0,TL1,Bar,2017.0,France,Domaine Lafage Bastide Miraflors Côtes du Rous...,Syrah/Granache,1.0,NaN,$14.25,Red,11/7/2020
1,TL2,Bar,2017.0,Italy,Scaia Paradiso,Corvina/Rondinella/Cabernet,1.0,4.0,$19.00,Red,5/30/2020
2,TL3,Bar,2016.0,Spain,Nubori Crianza Rioja,Tempranillo,1.0,4.0,$16.00,Red,11/23/2019
3,TL3,Bar,2016.0,Spain,Cyan Toro,Tempranillo,1.0,NaN,$16.50,Red,11/7/2020
4,TL4,Bar,2016.0,California,Sterling,Merlot,1.0,4.0,$-,Red,NaN


### Handling Missing Values

In [134]:
df.drop(df[df['Wine Name'].isna() == True].index, inplace=True)

In [135]:
df.isna().sum()

Location Code           0
Location                0
Vintage                 3
Region                  0
Wine Name               0
Grape/Blend             0
Quantity                0
Rating                 37
Unit Purchase Price     0
Type                    0
Purchase Date          12
dtype: int64

In [136]:
df['Vintage'] = df['Vintage'].fillna(0)

In [137]:
df = df.astype({'Quantity':'int', 'Vintage':'int'}) 

In [138]:
df.loc[68, 'Quantity'] = 0 # Mara - Drank last bottle
df.loc[23, 'Quantity'] = 1 # Chateau Labastide Orliac drank one

### Splitting and Encoding

In [163]:
# creating the training set
train = df[['Vintage', 'Region', 'Grape/Blend', 'Type', 'Rating']]
train = train.dropna(axis=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 1 to 92
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vintage      56 non-null     int32  
 1   Region       56 non-null     object 
 2   Grape/Blend  56 non-null     object 
 3   Type         56 non-null     object 
 4   Rating       56 non-null     float64
dtypes: float64(1), int32(1), object(3)
memory usage: 2.4+ KB


In [195]:
X = train[['Vintage', 'Region', 'Grape/Blend', 'Type']]
y = train[['Rating']]

### Visualization